# Proyecto #8 - Haciendo Análisis de Funnels para un Ecommerce con 6 Data Sets - Parte 2

Cool T-Shirts te ha preguntado si puedes analizar los datos de su sitio web. Tu trabajo es construir un "Funnel" (una descripcion de como las personas van al siguiente paso dentro de un proceso de multiples pasos)

En este caso, nuestro funnel seria como el siguiente proceso:

- 1. Un user visita la pagina web de Cool T-shirt
- 2. El usuario añade una prenda (camiseta) a su carrito de compras
- 3. El usuario hace click en "checkout"
- 4. El usuario compra la camiseta

### Ciudado: en el momento de usar el metodo `read_csv` pase un segundo parametro para parsear las fechas asi: `parse_dates=[1]`

In [1]:
import pandas as pd
purchase = pd.read_csv('datasets/purchase.csv', parse_dates=[1])
checkout = pd.read_csv('datasets/checkout.csv', parse_dates=[1])
cart = pd.read_csv('datasets/cart.csv', parse_dates=[1])
visits = pd.read_csv('datasets/visits.csv', parse_dates=[1])

In [2]:
purchase.head()

,user_id,purchase_time
0,4b44ace4-2721-47a0-b24b-15fbfa2abf85,2017-05-11 04:25:00
1,02e684ae-a448-408f-a9ff-dcb4a5c99aac,2017-09-05 08:45:00
2,4b4bc391-749e-4b90-ab8f-4f6e3c84d6dc,2017-11-20 20:49:00
3,a5dbb25f-3c36-4103-9030-9f7c6241cd8d,2017-01-22 15:18:00
4,46a3186d-7f5a-4ab9-87af-84d05bfd4867,2017-06-11 11:32:00


In [3]:
checkout.head()

,user_id,checkout_time
0,d33bdc47-4afa-45bc-b4e4-dbe948e34c0d,2017-06-25 09:29:00
1,4ac186f0-9954-4fea-8a27-c081e428e34e,2017-04-07 20:11:00
2,3c9c78a7-124a-4b77-8d2e-e1926e011e7d,2017-07-13 11:38:00
3,89fe330a-8966-4756-8f7c-3bdbcd47279a,2017-04-20 16:15:00
4,3ccdaf69-2d30-40de-b083-51372881aedd,2017-01-08 20:52:00


In [4]:
cart.head()

,user_id,cart_time
0,2be90e7c-9cca-44e0-bcc5-124b945ff168,2017-11-07 20:45:00
1,4397f73f-1da3-4ab3-91af-762792e25973,2017-05-27 01:35:00
2,a9db3d4b-0a0a-4398-a55a-ebb2c7adf663,2017-03-04 10:38:00
3,b594862a-36c5-47d5-b818-6e9512b939b3,2017-09-27 08:22:00
4,a68a16e2-94f0-4ce8-8ce3-784af0bbb974,2017-07-26 15:48:00


In [5]:
visits.head()

,user_id,visit_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00


Combine `visits` y `cart` usando un left merge

In [6]:
visits_cart = pd.merge(visits, cart, how='left')
visits_cart.head()

,user_id,visit_time,cart_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00
3,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:49:00
4,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT


Que tan largo es la nuevo dataframe?

In [7]:
visits_cart_rows = len(visits_cart)
visits_cart_rows

2052

Cuantos `timestamps` son `null` o `NaT` para la columna `cart_time`?

In [8]:
null_cart_time = len(visits_cart[visits_cart.cart_time.isnull()])
null_cart_time

1652

Que porcentaje de usuarios que han visitado el sitio han terminado poniendo una camiseta en su carrito de compras?

Nota: para calcular porcentajes, es util cambiar cualquier numero entero en un flotante `float()`. De otra forma Python suara una division entre enteros, el cual corta los puntos decimales

In [14]:
print(float(1-(null_cart_time/visits_cart_rows)))

0.1949317738791423


Repita el left merge para `cart` y `checkout` y cuente los valores `null`. Que porcentaje de usuarios pusieron items en el carrito de comoras, pero no procedieron a terminar la compra?

In [16]:
cart_checkout = pd.merge(cart, checkout, how='left')

In [30]:
cart_checkout_rows = len(cart_checkout)
null_cart_checkout = len(cart_checkout[cart_checkout.checkout_time.isnull()])
print(float(1-(null_cart_checkout/cart_checkout_rows)))
print(cart_checkout_rows)
print(null_cart_checkout)

0.7906976744186046
602
126


Haga un merge sobre todos pasos del funnel usando una serie de left merges. Guarde el resultado en la variable `all_data`

In [18]:
all_data = visits.merge(cart, how='left').merge(checkout, how='left').merge(purchase, how='left')
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:49:00,NaT,NaT
4,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT


Que porcentaje de usuarios procedieron al checkout, pero no comparon una camistea?

In [32]:
checkout_purchase = pd.merge(checkout, purchase, how='left')
checkout_purchase_rows = len(checkout_purchase)
null_purchase_time = len(checkout_purchase[checkout_purchase.purchase_time.isnull()])
print(float(1-(null_purchase_time/checkout_purchase_rows)))
print(null_purchase_time)
print(checkout_purchase_rows)

0.8311036789297659
101
598


Usando el dataframe gigante que guardamos en `all_data`, vamos a crear erl tiempo promedio desde la visita inicial hasta la compra final. Empice añadiendo la siguiente columna a su dataframe:

`
all_data['time_to_purchase'] = \
    all_data.purchase_time - \
    all_data.visit_time
`

In [27]:
all_data['time_to_purchase'] = all_data.purchase_time - all_data.visit_time
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time,time_to_purchase
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00,00:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT,NaT
3,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:49:00,NaT,NaT,NaT
4,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT,NaT


Examine el resultado usando

In [28]:
print(all_data.time_to_purchase)

0           NaT
1      00:44:00
2           NaT
3           NaT
4           NaT
         ...   
2589        NaT
2590        NaT
2591        NaT
2592        NaT
2593        NaT
Name: time_to_purchase, Length: 2594, dtype: timedelta64[ns]


Calcule el tiempo promedio de compra

In [29]:
print(all_data.time_to_purchase.mean())

0 days 00:44:02.672413
